In [24]:
from sklearn import datasets
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

In [33]:
class IrisDataset(Dataset):
    def __init__(self):
        iris = datasets.load_iris()
        feature = pd.DataFrame(iris.data, columns=iris.feature_names)
        target = pd.DataFrame(iris.target, columns=['target'])
        iris_data = pd.concat([target, feature], axis=1)
        
        self.x = torch.from_numpy(np.array(iris_data)[:, 1:].astype(np.float32))
        self.y = torch.from_numpy(np.array(iris_data)[:, :1].astype(np.longlong).flatten())
        self.n_samples = self.x.shape[0]
        
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples
        
iris_data = IrisDataset()



dataset = IrisDataset()


batch_size = 4
# create data spliter
def dataSplit(dataset, val_split=0.25, shuffle=False, random_seed=0):

    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    
    train_indices, val_indices = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    return train_sampler, valid_sampler

# base split parameters
val_split = 0.25
shuffle_dataset = True
random_seed= 42

train_sampler, valid_sampler = dataSplit(dataset=dataset, val_split=val_split, shuffle=shuffle_dataset, random_seed=random_seed)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]
     sepal length (cm)  sepal width (cm)

In [27]:
input_size = 4
hidden_size = 24
output_size = 3

class NeuralNet(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        # define first layer
        self.l1 = nn.Linear(input_size, hidden_size)
        # activation function
        self.relu = nn.ReLU()
        # define second layer
        self.l2 = nn.Linear(hidden_size, hidden_size)
        # define third layer
        self.l3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)

        return out


model = NeuralNet(input_size, hidden_size, output_size)

In [35]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 10

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (datas, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(datas)
        loss = criterion(outputs, labels)
        
        loss.backward()

        optimizer.step()

        if (i + 1) % 29 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1/10, step 29/29, loss = 0.0038
epoch 2/10, step 29/29, loss = 0.0209
epoch 3/10, step 29/29, loss = 0.0000
epoch 4/10, step 29/29, loss = 3.1092
epoch 5/10, step 29/29, loss = 0.0034
epoch 6/10, step 29/29, loss = 0.5098
epoch 7/10, step 29/29, loss = 0.0000
epoch 8/10, step 29/29, loss = 0.0003
epoch 9/10, step 29/29, loss = 0.0030
epoch 10/10, step 29/29, loss = 0.0000


In [30]:
filePath = "./model_test.pt"
torch.save(model, filePath)

In [36]:
model_new = torch.load(filePath)